# Creating songs theme and sentiment with chatgpt batch of jsol files

In [4]:
import re
import os
import sys
import json
import openai
import warnings
import numpy as np
import pandas as pd

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
sys.path.append('/home/or/dev/latin_music_statistics/crawler/data_dir')
sys.path.append(os.path.abspath(os.path.join(parent_dir, 'statistics_dir')))

from config import OPENAI_MODEL, THEME_PROMPT, SENTIMENT_PROMPT, BATCH_THEME_JSOL_FILE, BATCH_SENTIMENT_JSOL_FILE, DATA_JSOL_DIR, FINAL_DATA_FILE
df = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/processed_data.csv', encoding='utf-8')

In [76]:
# DATA_JSOL_DIR = os.path.join(DATA_DIR, 'jsol')

# BATCH_THEME_JSOL_FILE = os.path.join(DATA_JSOL_DIR, 'batch_song_theme')
# BATCH_SENTIMENT_JSOL_FILE = os.path.join(DATA_JSOL_DIR, 'batch_song_sentiment')

In [116]:
def creare_jsol_batch_file(base_file_name, openai_model, prompt, batch_size):
    num_batches = int(np.ceil(len(df) / batch_size))
    
    for i in range(num_batches):
        batch_df = df.iloc[i * batch_size:(i + 1) * batch_size]
        batch_file_name = f'{base_file_name}_{i + 1}.jsonl'
        
        with open(batch_file_name, 'w') as outfile:
            for ii, (idx, row) in enumerate(batch_df.iterrows()):
                openai_row = {
                    "custom_id": f"request-{ii + i * batch_size}",
                    "method": "POST",
                    "url": "/v1/chat/completions",
                    "body": {
                        "model": openai_model,
                        "messages": [
                            {
                                "role": "user",
                                "content": prompt.format(song=row['clean_lyrics'])
                            }
                        ]
                    }
                }
                outfile.write(json.dumps(openai_row) + '\n')

In [117]:
creare_jsol_batch_file(BATCH_THEME_JSOL_FILE, OPENAI_MODEL, THEME_PROMPT, OPENAI_BATCH_SIZE)
creare_jsol_batch_file(BATCH_SENTIMENT_JSOL_FILE, OPENAI_MODEL, SENTIMENT_PROMPT, OPENAI_BATCH_SIZE)

In [86]:
# # Theme

# creare_jsol_batch_file(batch_file_name, openai_model, prompt):

# with open(BATCH_THEME_JSOL_FILE, 'w') as outfile:
#     for ii, (idx, row) in enumerate(df.iterrows()):
#         # Create the JSON object
#         openai_row = {
#             "custom_id": f"request-{ii}",
#             "method": "POST",
#             "url": "/v1/chat/completions",
#             "body": {
#                 "model": OPENAI_MODEL,
#                 "messages": [
#                     {
#                         "role": "user",
#                         "content": THEME_PROMPT + row['clean_lyrics']
#                     }
#                 ]
#             }
#         }
        
#         outfile.write(json.dumps(openai_row) + '\n')

In [87]:
# # Sentiment

# with open(BATCH_SENTIMENT_JSOL_FILE, 'w') as outfile:
#     for ii, (idx, row) in enumerate(df.iterrows()):
#         # Create the JSON object
#         openai_row = {
#             "custom_id": f"request-{ii}",
#             "method": "POST",
#             "url": "/v1/chat/completions",
#             "body": {
#                 "model": OPENAI_MODEL,
#                 "messages": [
#                     {
#                         "role": "user",
#                         "content": SENTIMENT_PROMPT.format(song=row['clean_lyrics'])
#                     }
#                 ]
#             }
#         }
        

#         outfile.write(json.dumps(openai_row) + '\n')

In [118]:
def create_batch(batches_dir):
    batch_list = []
    
    # Loop through all files in the directory
    for filename in os.listdir(batches_dir):
        if filename.endswith(".jsonl"):
            # Full path to the file
            file_path = os.path.join(batches_dir, filename)
            
            # Upload the file and append the result to the list
            batch_file_status = client.files.create(file=open(file_path, "rb"), purpose="batch")

            batch_list.append(batch_file_status)

    return batch_list

load_batch_list = create_batch(DATA_JSOL_DIR)

In [ ]:
import time

def run_batch(batch_list):
    batch_run_status_list = []

    for single_batch in batch_list:
        batch_status = client.batches.create(
            input_file_id=single_batch.id,
            endpoint="/v1/chat/completions",
            completion_window="24h",
            metadata={
              "description": "nightly eval job"
            }
        )
    
        batch_run_status_list.append(batch_status)
        time.sleep(1)

    return batch_run_status_list

batch_run_status_list = run_batch(batch_list)

In [238]:
# iF the previous failed - then run this to get how man ran successfully
matching_files = []
seen_filenames = set()  # A set to track filenames that have already been added
batches_retrieve_list = client.batches.list()

# Loop through the batches_list
for batch in batches_retrieve_list:
    # Check if the batch status is 'completed'
    if batch.status == 'completed':
        # Get the input_file_id from the batch
        input_file_id = batch.input_file_id
        
        # Loop through the batch_list to find the matching file
        for file_obj in load_batch_list:
            if file_obj.id == input_file_id:
                # Check if the filename has already been added to matching_files
                if file_obj.filename not in seen_filenames:
                    # Record the matching input_file_id, filename, and output_file_id
                    matching_files.append({
                        'input_file_id': input_file_id,
                        'filename': file_obj.filename,
                        'output_file_id': batch.output_file_id
                    })
                    # Add the filename to the seen set
                    seen_filenames.add(file_obj.filename)

print(len(matching_files))

# # matching_files now contains all the matched files with 'completed' status without duplicates
# for match in matching_files:
#     print(f"input_file_id: {match['input_file_id']}, filename: {match['filename']}, output_file_id: {match['output_file_id']}")


1686


In [239]:
# # Rerun only failed batches running  - run the all faile/unran files
# # Step 1: Extract the filenames from matching_files
# processed_filenames = set([item['filename'] for item in matching_files])

# # Step 2: Filter batch_list to include only files not in processed_filenames
# filtered_batch_list = [file_obj for file_obj in load_batch_list if file_obj.filename not in processed_filenames]

# # Step 3: Run the run_batch function with the filtered list
# print(len(filtered_batch_list))
# if filtered_batch_list:
#     batch_run_status_list = run_batch(filtered_batch_list)
#     # Now batch_run_status_list contains the statuses of the newly run batches
# else:
#     print("All files have already been processed.")

In [240]:
# import pickle

# with open('batch_list.pkl', 'wb') as outfile:
#     pickle.dump(batch_list, outfile)

# with open('batch_run_status_list.pkl', 'wb') as outfile:
#     pickle.dump(batch_run_status_list, outfile)

In [313]:
# # Match raw the response
# for record in matching_files:
#     file_response = client.files.content(record['output_file_id'])
#     raw_response = json.loads(file_response.text)['response']['body']['choices'][0]['message']['content']
#     record['raw_response'] = raw_response

# # Function to extract the numeric part from the filename
# def extract_number(filename):
#     match = re.search(r'(\d+)\.jsonl$', filename)
#     return int(match.group(1)) if match else 0

# # Sort the list based on the numeric part of the filename
# matching_files_sorted = sorted(matching_files, key=lambda x: extract_number(x['filename']))

# # Split into two lists based on the type in the filename
# theme_records_list = [file for file in matching_files_sorted if 'theme' in file['filename']]
# sentiment_records_list = [file for file in matching_files_sorted if 'sentiment' in file['filename']]

# clean_theme_response_list = []
# clean_sentiment_response_list_raw = []

# for record in theme_records_list:
#     clean_theme_response_list.append(record['raw_response'].replace("The song theme is:", "").strip())

# for record in sentiment_records_list:
#     clean_sentiment_response_list_raw.append(eval(record['raw_response'].replace("json", "").replace("```","").strip()))


# # Function to normalize and extract sentiment values
# def normalize_and_extract(entry):
#     # Handle entries with 'sentiment' or 'sentiment_analysis' keys
#     if 'sentiment' in entry:
#         sentiment = entry['sentiment']
#     elif 'sentiment_analysis' in entry:
#         sentiment = entry['sentiment_analysis']
#     elif 'overall_sentiment' in entry:
#         sentiment = entry['overall_sentiment']
#     else:
#         sentiment = entry

#     # Normalize values: divide by 100 if they are greater than 1
#     normalized_sentiment = {key: value / 100 if value > 1 else value for key, value in sentiment.items()}
    
#     return normalized_sentiment

# # Apply the normalization function to each dictionary in the list
# clean_sentiment_response_list = [normalize_and_extract(entry) for entry in clean_sentiment_response_list_raw]

# df['theme'] = clean_theme_response_list
# df['sentiment'] = clean_sentiment_response_list

# df.to_csv(FINAL_DATA_FILE)

In [8]:
df = pd.read_csv('/home/or/dev/latin_music_statistics/crawler/data_dir/final_data.csv')

In [12]:
df.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'],axis=1,inplace=True)